## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Xiaoweizhai,CN,26.19,107.51,71.71,75,2,3.53,clear sky
1,1,Pevek,RU,69.70,170.31,35.04,81,100,19.08,overcast clouds
2,2,Castro,BR,-24.79,-50.01,51.66,82,0,1.57,clear sky
3,3,Hay River,CA,60.82,-115.80,82.45,33,0,5.55,clear sky
4,4,Poum,NC,-20.23,164.02,68.40,64,9,9.33,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [23]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Hay River,CA,60.82,-115.80,82.45,33,0,5.55,clear sky
10,10,Elhovo,BG,42.17,26.57,86.14,40,14,1.99,few clouds
14,14,Acarau,BR,-2.89,-40.12,77.45,74,8,7.78,clear sky
15,15,Hithadhoo,MV,-0.60,73.08,81.68,69,100,5.19,overcast clouds
16,16,Alofi,NU,-19.06,-169.92,76.89,73,40,11.50,scattered clouds
19,19,Celestun,MX,20.87,-90.40,86.25,68,23,16.89,few clouds
20,20,Kavieng,PG,-2.57,150.80,82.67,78,28,10.56,light rain
23,23,Tecoanapa,MX,16.52,-98.75,87.42,64,41,10.04,scattered clouds
27,27,Georgetown,MY,5.41,100.34,80.89,88,20,3.44,few clouds
33,33,Butaritari,KI,3.07,172.79,82.90,71,35,4.99,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                251
City                   251
Country                251
Lat                    251
Lng                    251
Max Temp               251
Humidity               251
Cloudiness             251
Wind Speed             251
Current Description    251
dtype: int64

In [27]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# no nans to drop
# issue with 'count' keeps telling me there are filled rows when they are empyt try to fix wit Nan
import numpy as np
NaN = np.nan

In [28]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = NaN
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Hay River,CA,82.45,clear sky,60.82,-115.80,NaN
10,Elhovo,BG,86.14,few clouds,42.17,26.57,NaN
14,Acarau,BR,77.45,clear sky,-2.89,-40.12,NaN
15,Hithadhoo,MV,81.68,overcast clouds,-0.60,73.08,NaN
16,Alofi,NU,76.89,scattered clouds,-19.06,-169.92,NaN
19,Celestun,MX,86.25,few clouds,20.87,-90.40,NaN
20,Kavieng,PG,82.67,light rain,-2.57,150.80,NaN
23,Tecoanapa,MX,87.42,scattered clouds,16.52,-98.75,NaN
27,Georgetown,MY,80.89,few clouds,5.41,100.34,NaN
33,Butaritari,KI,82.90,scattered clouds,3.07,172.79,NaN


In [29]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    #6c. Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [38]:
#check data frame
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Hay River,CA,82.45,clear sky,60.82,-115.80,Ptarmigan Inn
10,Elhovo,BG,86.14,few clouds,42.17,26.57,Slavey Homestay
14,Acarau,BR,77.45,clear sky,-2.89,-40.12,castelo pizzaria
15,Hithadhoo,MV,81.68,overcast clouds,-0.60,73.08,Scoop Guest House
16,Alofi,NU,76.89,scattered clouds,-19.06,-169.92,Taloa Heights
19,Celestun,MX,86.25,few clouds,20.87,-90.40,Posada Lilia
20,Kavieng,PG,82.67,light rain,-2.57,150.80,Nusa Island Retreat
23,Tecoanapa,MX,87.42,scattered clouds,16.52,-98.75,Hotel Manglares
27,Georgetown,MY,80.89,few clouds,5.41,100.34,Cititel Penang
33,Butaritari,KI,82.90,scattered clouds,3.07,172.79,Isles Sunset Lodge


In [39]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].isnull().values.sum()
clean_hotel_df = hotel_df.dropna(how='any')
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Hay River,CA,82.45,clear sky,60.82,-115.80,Ptarmigan Inn
10,Elhovo,BG,86.14,few clouds,42.17,26.57,Slavey Homestay
14,Acarau,BR,77.45,clear sky,-2.89,-40.12,castelo pizzaria
15,Hithadhoo,MV,81.68,overcast clouds,-0.60,73.08,Scoop Guest House
16,Alofi,NU,76.89,scattered clouds,-19.06,-169.92,Taloa Heights
19,Celestun,MX,86.25,few clouds,20.87,-90.40,Posada Lilia
20,Kavieng,PG,82.67,light rain,-2.57,150.80,Nusa Island Retreat
23,Tecoanapa,MX,87.42,scattered clouds,16.52,-98.75,Hotel Manglares
27,Georgetown,MY,80.89,few clouds,5.41,100.34,Cititel Penang
33,Butaritari,KI,82.90,scattered clouds,3.07,172.79,Isles Sunset Lodge


In [41]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [46]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [51]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.75)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))